In [1]:
import math
import tsplib95
import statistics
import numpy as np
from itertools import groupby
from scipy.special import _logsumexp

In [2]:
def first_index(arr: list) -> float:
    if (len(arr) == 1):
        return arr
    else:
        return arr[0]

In [3]:
def decide_city(Vt: float, acc: float, time: float, eye: int, n: int) -> int:
    city = 0
    if eye == 1:
        divided_value = int(Vt * np.nan_to_num(time) - (1/2) * acc * (np.power(np.nan_to_num(time), 2)))
    else:
        divided_value = int(Vt * np.nan_to_num(time) + (1/2) * acc * (np.power(np.nan_to_num(time), 2)))
        
    if divided_value > (n - 1):
        city = divided_value - (int(divided_value / n) * n) 

    return city

### Normal functions of the algorithm

In [4]:
def herding(route: list, Vt: list, fit: list, n: int, dimension: int, acc: list, time: list) -> list:
    
    route_copy = np.array(route)
    fit1 = np.sort(fit, 0, kind="heap")
    idx = np.argsort(fit, 0, kind="heap")
    route1 = np.zeros(len(route))
    Vt1 = np.zeros((n, dimension))
    acc1 = np.zeros((n, dimension))
    time1 = np.zeros((n, dimension))

    for i in range(n):
        route1[i] = first_index(route_copy[idx[i]])
        Vt1[i, :] = Vt[idx[i][0], :]
        acc1[i, :] = acc[i, :]
        time1[i] = time[i]
    
    return route1, Vt1, fit1, acc1, time1

In [5]:
def update_v(Vt: list, n: int, dimension: int, acc: list, time: list, route: list, fit: list, eye: int) -> list:
    #Choosing left and right dogs
    right_dog = np.random.randint(2,3)
    if (right_dog == 2):
        left_dog = 3
    else:
        left_dog = 2

    acc1 = np.ones((n,dimension))
    time1 = np.ones((n,1))
    Vt1 = np.copy(Vt)
    r = len(acc)
    l = len(acc)
    acc2 = np.zeros((r, l))
    fit1 = fit[l-1]

    #Finding Dg value to choose which sheep to gather and which to stalk
    fit2 = (fit[1] + fit[2]) / 2
    f = 0
    tempg = 0
    temps = 0

    #Setting parameters for eyeing
    if eye == 1:
        if fit[right_dog] < fit[left_dog]:
            acc2[left_dog, :] = -1 * acc[left_dog, :]
            f = left_dog
        else:
            acc2[right_dog, :] = -1 * acc[right_dog, :]
            f = right_dog

    for i in range(n):
        for j in range(dimension):
            #Velocity updation of dogs
            if (i <= 2):
                Vt1[i][j] = math.sqrt(np.power(Vt[i][j], 2) + (2 * acc[i][j]) * abs(route[i]))
                
            #Velocity updation of sheep
            if (i > 2):
                if eye == 1:
                    Vt1[i][j] = math.sqrt(np.power(Vt1[f][j], 2) + (2*acc2[f][j]) * abs(route[i]))

                else:
                    #Velocity updation of gathered sheep
                    if ((fit1 - fit[i]) > (fit2 - fit[i])):
                        Vt1[i][j] = math.sqrt(np.power(Vt1[0,j], 2) + (2*acc[0,j]))
                        #tempg[i] = i        
                
                    #Velocity updation of stalked sheep
                    if ((fit1 - fit[i]) <= (fit2 - fit[i])):
                        Vt1[i][j] = math.sqrt(np.power(Vt[right_dog][j]*math.tan(np.random.randint(1,89)), 2) + (2*acc[right_dog][j]*abs(route[right_dog]))) + math.sqrt(np.power(Vt[left_dog][j]*math.tan(np.random.randint(91,179)), 2) + (2*acc[left_dog][j]*abs(route[left_dog])))
                        Vt1[i][j] = (Vt1[i][j]) / 2
                        #temps[i] = i
    
    #Updation of time and acceleration
    for i in range(n):
        s = 0
        for j in range(dimension):
            acc1[i][j] = abs((Vt1[i][j]) - (Vt[i][j])) / (time[i][j])
            s += (_logsumexp.logsumexp(Vt1[i][j]) - _logsumexp.logsumexp(Vt[i][j])) / acc1[i][j]
            
        time1[i] = abs(np.mean(s))

    return Vt1, acc1, time1, right_dog, left_dog, tempg, temps

In [6]:
def check(route: list, n: int, dimension: int, acc: list, Vt: list, time: list, objf: object) -> list:
    route1 = np.copy(route)
    acc1 = np.copy(acc)
    time1 = np.copy(time)
    Vt1 = np.copy(Vt)
    for i in range(n):
        for j in range(dimension):
            if (route[i] >= len(objf[0]) or route[i] <= np.min(objf[0])):
                route1[i] = np.random.randint(np.min(objf[0]), len(objf[0]))
                acc1[i][j] = np.random.random()
                time1[i] = np.random.random()
    
    for i in range(n):
        for j in range(dimension):
            if (math.isnan(acc[i][j]) == 1 or acc[i][j] == 0):
                route1[i] = np.random.randint(np.min(objf[0]), len(objf[0]))
                acc1[i][j] = np.random.random()
                time1[i] = np.random.random()

    for i in range(n):
        for j in range(dimension):
            if (math.isnan(Vt[i][j]) == 1 or Vt[i][j] == 0):
                route1[i] = np.random.randint(np.min(objf[0]), len(objf[0]))
                acc1[i][j] = np.random.random()
                time1[i] = np.random.random()

    for i in range(n):
        for j in range(dimension):
            if (math.isnan(time1[i]) == 1 or time1[i] == 0):
                route1[i] = np.random.randint(np.min(objf[0]), len(objf[0]))
                acc1[i][j] = np.random.random()
                time1[i] = np.random.random()


    return route1, acc1, time1, Vt1

### Discretization of the problem

In [7]:
def discrete_fitness(route: list, n: int, objf: object):
    fit1 = np.zeros((len(route), 1))

    for i in range(len(route)):    
        if (i != len(route) - 1):
            fit1[i] = objf[int(route[i])][int(route[i+1])]
        else:            
            fit1[i] = objf[int(route[i])][int(route[0])]
    
    #best fit and position
    best_fit = np.min(fit1)
    pos = np.argmin(fit1)
    
    return fit1, best_fit, pos 

In [8]:
def discrete_generate(n: int, dimension: int) -> list:
    boundary_no = 1
    pop = np.zeros((n, dimension))

    if boundary_no == 1:
        pop = np.random.randint(0, 1+1, size=(n,dimension))

    x = np.random.random(size = (n, dimension))
    
    return pop, x

In [9]:
def discrete_update(Vt: list, time: list, acc: list, n: int, dimension: int, eye: int) -> list:
    route1 = np.zeros((n, dimension))
    for i in range(n):
        for j in range(dimension):
            #Updating the position of dogs
            # Check the min and max value of each variable in the equation
            if (i <= 2):
                route1[i] = decide_city(Vt[i][j], acc[i][j], time[i], eye, n)
                
            #Updating position of sheep
            if (i > 2):
                if eye == 1:
                    route1[i] = decide_city(Vt[i][j], acc[i][j], time[i], eye, n)
                else:
                    route1[i] = decide_city(Vt[i][j], acc[i][j], time[i], eye, n)    

    return route1

### Funtions to define  the route

In [10]:
# Need to put at least one sheep per city
# Create a function to make the reward strategy

def city_choice(n: int) -> list:
    route = []
    city = []
    sheep = np.random.randint(0, n)

    for map in range(n):
        city.append(map)
    
    while len(city) != 0:
        if sheep not in city:
            sheep = np.random.randint(0, n)
        else:
            route.append(sheep)
            city.remove(sheep)

    return route

# Create a random number each iteration and after that made a line of the matrix receive that number
def generate_city(n: int, dim: int) -> list:
    city_matrix = np.zeros((n, dim))
    sheep = city_choice(n)

    for i in range(n):
        for j in range(dim):
            if (sheep[i] == j):
                city_matrix[i][j] = 1
    
    x = np.random.random(size = (n, dim))
    
    return city_matrix, x

# Debug function
# There's no need to use this function in the main algorithm, just in the test file
def print_position_of_sheep(city_matrix: list):
    for i in range(len(city_matrix)):
        for j in range(len(city_matrix)):
            if (city_matrix[i][j] != 0):
                print("Position -> ",j)

# Function to create route
def create_route(city_matrix: list) -> list:
    route = []
    for i in range(len(city_matrix)):
        for j in range(len(city_matrix)):
            if (city_matrix[i][j] == 1):
                route.append(j)
    print("Route -> ",route,"\n")

    return route

def catch_city(route: list, n: int) -> list:
    route1 = np.zeros(n)
    for i in range(len(route)):
       route1[i] = route[i][0]
    
    return route1

### Creation of a function to select the cities that were not chosen

In [25]:
def city_swap(route: list, objf: object):
    count = 0
    same_city = []
    city_position = []
    not_in_route = []
    # Do a mini heuristic based on the nn
    print("Route before loop -> ", route,"\n")
    for i in range(len(route)):
        j = i+1
        while j <= len(route) - 1:    
            if (route[i] == route[j]):
                same_city.append(route[j])
                city_position.append(j)
                count += 1
            j+=1
    
    # Verify what cities that are not in the route
    for i in range(len(route)):
        if i not in route:
            not_in_route.append(i)
    not_in_route = np.sort(not_in_route)
    
    for i in range(len(city_position)):
        j = i+1
        while j <= len(city_position) - 1:    
            if (city_position[i] == city_position[j]):
                city_position.pop(j)
            j+=1
    print("Not in rout sorted -> ", not_in_route)
    print("\nSame city -> ", same_city,"\nCity position -> ", city_position,"\n")    

### Application of TSP

In [26]:
problem = tsplib95.load('../data/domain/bays29.tsp')

In [27]:
#Maximum no. of iterations
gen = 200

#Population size
n = problem.dimension

#Optimization function name
dim = problem.dimension

#Intialize the population(init_p-Population,acc-acceleration of each individual)
init_p, acc = generate_city(n, dim)

#Vt = velocity of each individuals
Vt = np.zeros((n, dim))

#Time of each individual
time = np.random.random(size = (n,1))

#Max fitness value
fopt = math.inf
#Variable to store fitness
fit = np.zeros((n, 1))
pop = init_p
#print_position_of_sheep(pop)
#k = counter variable for iterations required for Eyeing mechanism
k = 1
fopt_1 = np.zeros((gen, 1))
# Generate a route and at the same time verify if this route is valid
# Needs to generate before the loop and adjust in till the iterations turns it valid
route = create_route(pop)

for g in range(gen):
    # Calculate fitness of indivuals
    # Need to remember to debug this function
    fit, maxf, pos = discrete_fitness(route, n, getattr(problem, 'edge_weights'))
    #print("Valid -> ", valid,"\n")
    eye = 0
    if g == 0:
        fopt = maxf
    
    #Finding the optimum fitness value
    if fopt > maxf:
        fopt = maxf
    
    fopt_1[g] = fopt
    if g > 0:
        if fopt_1[g] > fopt_1[g-1]:
            k += 1
            if k > 5:
                eye = 1
                k = 0
    
    route, Vt, fit, acc, time = herding(route, Vt, fit, n, dim, acc, time)
    
    Vt, acc, time, r1, l1, tempg, temps = update_v(Vt, n, dim, acc, time, route, fit, eye)
    
    route = discrete_update(Vt, time, acc, n, dim, eye)
    # Do a function to catch the first index of the array
    route = catch_city(route, n)
    # Checking if the range of the population is maintained
    route, acc, time, Vt = check(route, n, dim, acc, Vt, time, getattr(problem, 'edge_weights'))
    city_swap(route, getattr(problem, 'edge_weights'))
    print("Route -> ",route,"\n")

Route ->  [28, 4, 15, 19, 3, 5, 13, 6, 16, 12, 26, 23, 27, 8, 21, 25, 10, 11, 20, 18, 14, 1, 17, 9, 2, 22, 0, 7, 24] 

Route before loop ->  [ 1. 14. 21. 20.  2. 24. 18. 25.  1.  5. 26. 11. 15. 15. 23.  7. 21. 11.
 18. 19. 15. 15. 23. 16.  4.  4. 10. 19. 16.] 

Not in rout sorted ->  [ 0  3  6  8  9 12 13 17 22 27 28]

Same city ->  [1.0, 21.0, 18.0, 11.0, 15.0, 15.0, 15.0, 15.0, 15.0, 23.0, 19.0, 15.0, 16.0, 4.0] 
City position ->  [8, 16, 18, 17, 13, 20, 21, 22, 27, 28, 25] 

Route ->  [ 1. 14. 21. 20.  2. 24. 18. 25.  1.  5. 26. 11. 15. 15. 23.  7. 21. 11.
 18. 19. 15. 15. 23. 16.  4.  4. 10. 19. 16.] 

Route before loop ->  [ 7. 17. 20. 15. 22. 13. 27. 17. 26.  5. 24.  6.  8. 27. 25. 18. 26. 12.
  2.  2.  7. 15. 18.  5.  5. 17. 23. 28. 27.] 

Not in rout sorted ->  [ 0  1  3  4  9 10 11 14 16 19 21]

Same city ->  [7.0, 17.0, 17.0, 15.0, 27.0, 27.0, 17.0, 26.0, 5.0, 5.0, 27.0, 18.0, 2.0, 5.0] 
City position ->  [20, 7, 25, 21, 13, 28, 16, 23, 24, 22, 19] 

Route ->  [ 7. 17. 20. 15

C:\Users\artur\AppData\Local\Temp/ipykernel_15964/3957245458.py:60: RuntimeWarning: invalid value encountered in double_scalars
  s += (_logsumexp.logsumexp(Vt1[i][j]) - _logsumexp.logsumexp(Vt[i][j])) / acc1[i][j]


Route before loop ->  [ 4.  8. 19.  6.  5. 17. 19. 15. 15.  5. 10. 12. 20. 26.  1.  0.  7. 19.
 17.  3. 17.  9. 24. 27. 26.  1. 10.  0. 19.] 

Not in rout sorted ->  [ 2 11 13 14 16 18 21 22 23 25 28]

Same city ->  [19.0, 19.0, 19.0, 5.0, 17.0, 17.0, 19.0, 19.0, 15.0, 10.0, 26.0, 1.0, 0.0, 19.0, 17.0] 
City position ->  [6, 17, 28, 9, 18, 20, 8, 26, 24, 25, 27] 

Route ->  [ 4.  8. 19.  6.  5. 17. 19. 15. 15.  5. 10. 12. 20. 26.  1.  0.  7. 19.
 17.  3. 17.  9. 24. 27. 26.  1. 10.  0. 19.] 

Route before loop ->  [12.  6. 18.  1. 19.  8. 13.  2. 15. 10. 22. 23. 21.  1. 12.  5. 11.  9.
  1.  4.  3. 28. 23. 19. 18. 25. 13. 18. 13.] 

Not in rout sorted ->  [ 0  7 14 16 17 20 24 26 27]

Same city ->  [12.0, 18.0, 18.0, 1.0, 1.0, 19.0, 13.0, 13.0, 23.0, 1.0, 18.0, 13.0] 
City position ->  [14, 24, 27, 13, 18, 23, 26, 28, 22] 

Route ->  [12.  6. 18.  1. 19.  8. 13.  2. 15. 10. 22. 23. 21.  1. 12.  5. 11.  9.
  1.  4.  3. 28. 23. 19. 18. 25. 13. 18. 13.] 

Route before loop ->  [11. 27. 19

KeyboardInterrupt: 